In [1]:
import fsspec
from itertools import chain


from etoLib.eto_functions import grepfxn, unique

In [2]:
def _read_config(configurationfile):
    '''Read configuration file and parse out the inputs line by line'''

    # Note: if you want run rcp 8.5, then all you have to do is change the rcp_source parameter from within config file
    # It only affects grepfxn(rcp_source,all_files) below

    with open(configurationfile) as f:
        data = {}
        for line in f:
            key, value = line.strip().split(' = ')
            data[key] = value
    return(data)

In [3]:
# Note that the difference between historical and future outputs in cloud are based on these 2 configuration files.
# configurationfile = 'configurationfile_referenceET_test_future.ini'
configurationfile = 'configurationfile_referenceET_test_historical.ini'

data = _read_config(configurationfile)

print(data)

{'model_files': 's3', 'data_source': 'METDATA', 'output_folder': 'in', 'elevfile': 'in/Elevation/cgiar_srtmmerge.tif', 'tiffolder': 'C:/Users/GRouze/Desktop/ReferenceET_Python/Future-MACA/Individual/geotiffs', 'ET0_method': 'PM', 'ET0_winddat': 'yes', 'ET0_crop': 'short', 'to_clip': 'True', 'model': 'MIROC5', 'northmost': '42.54', 'southmost': '38.6', 'westmost': '-76.3', 'eastmost': '-74.23', 'pad_factor': '0.50', 'rcp_source': 'historical', 'bucket_filepath_root': 'dev-et-data/out/DelawareRiverBasin/', 'bucket_filepath_branch': 'Run08262020_Firstattemptcloudpush', 'MACA_start_bucket': 'dev-et-data/in/DelawareRiverBasin/'}


In [4]:
#! cp ../00-rouze-orig-notebooks/*.ini .

In [5]:
model_files = data['model_files']
data_source = data['data_source']
output_folder = data['output_folder']
elevfile = data['elevfile']
tiffolder = data['tiffolder']
ET0_method = data['ET0_method']
ET0_winddat = data['ET0_winddat']
ET0_crop = data['ET0_crop']
to_clip = data['to_clip']
model = data['model']
northmost = float(data['northmost'])
southmost = float(data['southmost'])
westmost = float(data['westmost'])
eastmost = float(data['eastmost'])
pad_factor = float(data['pad_factor'])
rcp_source = data['rcp_source']
MACA_start_bucket = data['MACA_start_bucket']

'''This is needed to retrieve the netCDF files from the dev-et-data AWS bucket'''
# os.chdir(model_files)
fs = fsspec.filesystem(model_files, anon=False, requester_pays=True)

all_files = fs.find(MACA_start_bucket)

# This prints all of the files in dev-et-data/in/DelawareRiverBasin/ or MACA_start_bucket...a big set of outputs, so skipped
# print(all_files)

In [6]:
all_files

['dev-et-data/in/DelawareRiverBasin/.ipynb_checkpoints/junk',
 'dev-et-data/in/DelawareRiverBasin/DRB_water_mask_inland.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/.ipynb_checkpoints/junk',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950001.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950002.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950003.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950004.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950005.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950006.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950007.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950008.tif',
 'dev-et-data/in/DelawareRiverBasin/ETo/1950/reference_evapotranspiration_1950009.tif',
 'dev-et-data/in

In [7]:
# THE CODE BELOW IS PARSED FROM THE CONDIITION WHEN DEALING WITH METDATA

In [8]:
# Split models apart that are to be used for ensemble averaging
models_parsed = [x.strip() for x in model.split(',')]

# Whittle down the number of files if the folder contains both rcp 4.5 and rcp 8.5 files
# Right now, the code can only handle one model of METDATA output (8/21/2020)
rcp_all_files = [grepfxn(rcp_source,all_files)][0]

# Iterate the files by each each specified model
models_list=[]
for i in range(len(models_parsed)):
    model_files_loop = [grepfxn(models_parsed[i],rcp_all_files)][0]
    models_list.append(model_files_loop)
        
# Flatten series of lists into one list
rcp_all_files = list(chain(*models_list))

# prints all netCDF files from 1950-2100 from MACA (radiation, precipitation, wind etc.)
print(rcp_all_files)


['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1970_1974_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1975_1979_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1980_1984_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1985_1989_CONUS_daily.nc', 'dev-et-data/in/DelawareRiverBasin/PPT/

In [9]:
rcp_all_files

['dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1970_1974_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1975_1979_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1980_1984_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBasin/PPT/Climatology/macav2metdata_pr_MIROC5_r1i1p1_historical_1985_1989_CONUS_daily.nc',
 'dev-et-data/in/DelawareRiverBa

In [10]:
# Find and compile the year blocks into a list
dfis=[]
for out in rcp_all_files:
    a=out.split('_')
    dfi = a[5]+'_'+a[6]
    dfis.append(dfi)

# print(dfis)
    
# Distill the above list into unique year blocks, as there will be duplicates from multiple climate inputs
year_all=unique(dfis);print(year_all)

# For prototyping only
year_block=0
# print(year_all)
# Print the first entry in the year list
print(year_all[year_block])

['1950_1954', '1955_1959', '1960_1964', '1965_1969', '1970_1974', '1975_1979', '1980_1984', '1985_1989', '1990_1994', '1995_1999', '2000_2004', '2005_2005']
1950_1954
